## URL Encoding

20.02.14  
  
Url을 Encoding한 후, url query에 city_id를 추가하는 작업

### Library

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from urllib import parse

### Function

Encoder

In [2]:
def change_url(before):
    # Url parsing
    url = parse.urlparse(before)
    
    # query를 파싱해서 딕셔너리로 반환
    query = parse.parse_qs(url.query)
    
    # Encoding
    result = parse.urlencode(query, doseq=True)
    return result

Text changer

In [3]:
def change_text(before, after, data):
    data = data.apply(lambda x: str(x).replace(before, after))
    return data

--- 

### Url Encoding

Load Data

In [4]:
# url 데이터 불러오기
url = pd.read_csv('url.csv', encoding = 'utf-8')

# 데이터 예시
list(url['before'].sample())

['https://findmybucketlist.com/search/newresult?keyword=페피콘&category=stay']

http://인 것과 https://인 것 구분

In [5]:
url['s'] = url['before'].apply(lambda x: 1 if 'https' in x else 0)

Encoding

In [6]:
# UTF-8 Encoding
url['after'] = url['before'].apply(lambda x : change_url(x))

# Encoding 결과
list(url['after'].sample())

['keyword=%EC%9D%B4%EC%95%84&category=stay']

연결자를 +가 아닌 %20으로 문자 변경

In [7]:
url['after'] = change_text('+', '%20', url['after'])

http:s//일 경우와 http://일 경우를 구분지어 url 생성

In [8]:
# url 생성
for i in range(len(url)):   
    if url['s'][i] == 1:
        # s가 있었던 경우에는 https~
        url['after'][i] = 'https://findmybucketlist.com/search/newresult?' + url['after'][i]
    else:
        # s가 없던 경우에는 http~
        url['after'][i] = 'http://findmybucketlist.com/search/newresult?' + url['after'][i]

# url 생성 결과   
list(url['after'].sample())

['https://findmybucketlist.com/search/newresult?keyword=%EC%84%B8%EB%B6%80&category=stay']

cityid를 붙이기 위한 'name' 뽑아내기  

In [9]:
url['name'] = url['before'].apply(lambda x: parse.parse_qsl(parse.urlparse(x).query)[0][1])

1. parse.urlparse(x).query = url을 parsing한 후 query만 뽑아낸다.  
2. parse.parse_qsl(...)[0][1] = 뽑아낸 query를 리스트로 만들고 첫번째의 value를 뽑아낸다.  

In [10]:
list(url['before'].sample())

['https://findmybucketlist.com/search/newresult?keyword=몸바사&category=stay']

↓ 뽑아내기

In [11]:
url['name'].sample(3)

3222    포르데노네
2062      아테네
1319    바르셀로나
Name: name, dtype: object

---  

### City_Id 붙이기

In [12]:
# city_id 데이터 불러오기 
city = pd.read_excel('city_id.xlsx')

# id가 존재하는 것만 남기기
city = city[city['id'].notnull()]

# city_id를 기준으로 중복 제거
city = city.drop_duplicates(['id'], keep = 'first').reset_index(drop = True)

# city_id 예시
city.sample(3)

,name,id
2434,마르키,-515659.0
1200,피사,-124918.0
2336,울란우데,-3024639.0


만들어진 City Id와 합치기

In [13]:
# url + city
url = pd.merge(url, city, how = 'left', on = 'name')

# 합쳐진 예시
url[['before', 'after', 'id', 'name']].sample(3)

,before,after,id,name
7340,https://findmybucketlist.com/search/newresult?...,https://findmybucketlist.com/search/newresult?...,-3033871.0,블라디보스토크
5721,https://findmybucketlist.com/search/newresult?...,https://findmybucketlist.com/search/newresult?...,-2435281.0,막탄
234,https://findmybucketlist.com/search/newresult?...,https://findmybucketlist.com/search/newresult?...,NaN,스키


id가 float으로 로드되었는데, 없는거 제외하고 int로 바꿔주기

In [14]:
# url['id']가 현재 float임
url['id'].dtypes

dtype('float64')

In [15]:
# nan이면 비워주고, 아니면 int로
url['id'] = url['id'].apply(lambda x: '' if str(x)=='nan' else str(int(x)))

최종 url 생성하기  
  
1. city_id가 있는 것은 **&city_id=id값**을 붙여준다.  
2. city_id가 없는 것은 그냥 놔둔다.  

In [16]:
url['result'] = 'not_yet'

for i in range(len(url)):
    if url['id'][i] == '':
        # id가 없다면 그냥 [after url]값 가져오기
        url['result'][i] = url['after'][i]
    else:
        # id가 있으면 [after url] + '&city_id' + [city_id]
        url['result'][i] = url['after'][i] + '&city_id=' + url['id'][i]

# result 결과
list(url['result'].sample())

['https://findmybucketlist.com/search/newresult?keyword=%EC%98%A4%EC%82%AC%EC%B9%B4&category=stay&city_id=-240905']

내보내기

In [17]:
url.to_excel('Url_result.xlsx', index = False)